In [28]:
import mne
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg') 

# Having Issues:68, 95, 113, 120, 137

for i in range(120, 121):
    print(f'\n=== Subject {i:03d} ===')

    # --- Load data ---
    raw = mne.io.read_raw_eeglab(
        f'rEEG/sub-{i:03d}/eeg/sub-{i:03d}_task-Rest_eeg.set',
        preload=True

    )

    # --- Remove fixed bad channels  ---
    bad_idx = [45, 4, 47]   # required removals
    bads_to_remove = [raw.ch_names[idx] for idx in bad_idx]
    if 'Resp' in raw.ch_names:
        raw.drop_channels(['Resp'])
    raw.drop_channels(bads_to_remove)

    # --- Montage ---
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    # --- LOF bad channel detection ---
    from mne.preprocessing import find_bad_channels_lof

    bads_lof = find_bad_channels_lof(
        raw,
        n_neighbors=7,
        threshold=2,
        picks='eeg'
    )
    raw.info['bads'] = bads_lof

    # --- Interpolate ---
    raw.interpolate_bads(reset_bads=True)

    # --- Remove EEGLAB boundaries BEFORE filtering ---
    if len(raw.annotations):
        mask = raw.annotations.description == 'boundary'
        if mask.any():
            raw.annotations.delete(mask)

    # --- Filter ---
    raw.filter(1., 45., fir_design='firwin')

    # --- Resample ---
    raw.resample(250, npad="auto")

    # --- Save ---
    raw.save(f'rEEG/sub-{i:03d}/preICA{i:03d}_raw.fif', overwrite=True)
    print(f'Saved pre-ICA data for subject {i:03d}')



=== Subject 120 ===
Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-120\eeg\sub-120_task-Rest_eeg.fdt
Reading 0 ... 90359  =      0.000 ...   180.718 secs...


LOF: Detected bad channel(s): ['C3', 'T7', 'TP9', 'TP10', 'Cz', 'T8', 'C5', 'TP7', 'TP8', 'C6']
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 50 sensor positions
Interpolating 10 sensors
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)



C:\Users\User\AppData\Local\Temp\ipykernel_24304\1122401242.py:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(


Writing c:\Users\User\Documents\EEG_Project\rEEG\sub-120\preICA120_raw.fif
Closing c:\Users\User\Documents\EEG_Project\rEEG\sub-120\preICA120_raw.fif
[done]
Saved pre-ICA data for subject 120
